In [94]:
import numpy as np
import pandas as pd
import math
import re

Wszystkie parzyste i co najwyzej w odleglosci 64

In [95]:
FILE_NAME = "test.txt"
#FILE_NAME = "input.txt"
DIAG=False
DIAG2 = False

In [96]:
CHAR_MAP = {".":0, "#":1,"S":2}

def getCharFromNum(num):
    for x,y in CHAR_MAP.items():
        if y == num:
            return x
            

In [97]:
def buildArrayFromLines(lines):
    
    tmp_arr = []
    for line in lines:
        tmp_line = []    
        
        for c in line:
            if c != "\n":
                tmp_line += [CHAR_MAP[c]]
        tmp_arr.append(tmp_line)
    
    
    return np.array(tmp_arr)
    

In [98]:
# OK
def parseFile(fileName):   
    with open(fileName) as file:
        lines =  file.readlines()    
   
    my_arr = buildArrayFromLines(lines)
    st_point = np.where(my_arr == CHAR_MAP["S"])
    if DIAG: print(st_point)
    
    my_arr[st_point[0],st_point[1]] = CHAR_MAP["."]
    
    
    return my_arr,st_point


In [99]:
def decideNextStep(cell,direct):
    if DIAG: print(cell)
    res = []    
    if cell == 0:
        res  = [direct]
    elif cell == 1:
        res = [(-direct[1],-direct[0])]
    elif cell == 2:
        res = [(direct[1],direct[0])]
    elif cell == 3:
        if direct[1] == 0:
            res  = [direct]
        else:
            res = [(-1,0),(1,0)]
    elif cell == 4:
        if direct[0] == 0:
            res  = [direct]
        else:
            res = [(0,-1),(0,1)]
    else:
        print("What the hell?")
    return res

assert decideNextStep(1,(0,1)) == [(-1,0)]
assert decideNextStep(1,(-1,0)) == [(0,1)]
assert decideNextStep(1,(1,0)) == [(0,-1)]
assert decideNextStep(1,(0,-1)) == [(1,0)]

assert decideNextStep(2,(0,1)) == [(1,0)]
assert decideNextStep(2,(-1,0)) == [(0,-1)]
assert decideNextStep(2,(1,0)) == [(0,1)]
assert decideNextStep(2,(0,-1)) == [(-1,0)]

assert decideNextStep(3,(0,1)) == [(-1,0),(1,0)]
assert decideNextStep(3,(-1,0)) == [(-1,0)]
assert decideNextStep(3,(1,0)) == [(1,0)]
assert decideNextStep(3,(0,-1)) ==  [(-1,0),(1,0)]

assert decideNextStep(4,(0,1)) == [(0,1)] #[(-1,0),(1,0)]
assert decideNextStep(4,(-1,0)) == [(0,-1),(0,1)]
assert decideNextStep(4,(1,0)) == [(0,-1),(0,1)]
assert decideNextStep(4,(0,-1)) ==  [(0,-1)] #[(0,-1),(0,1)]

assert decideNextStep(0,(0,1)) == [(0,1)]
assert decideNextStep(0,(-1,0)) == [(-1,0)]
assert decideNextStep(0,(1,0)) == [(1,0)]
assert decideNextStep(0,(0,-1)) == [(0,-1)]

In [100]:
(1,1)+(-1,-1)

(1, 1, -1, -1)

In [101]:
#STEPS_PROCESSED = set()
#STEPS_TO_PROCESS = set()

In [102]:
def processStep(inMap,outMap,x,y,direct,STEPS_PROCESSED,STEPS_TO_PROCESS):
    """
    direct is (,) and a direction we had coming into the cell
    """
    if DIAG: print(">> step to process:",x,y,direct)
    shape_X,shape_Y = inMap.shape
    if not(x in range(0,shape_X) and y in range(0,shape_X)):
        # we had stepped  off the map
        if DIAG: print("off the map")
        pass
    elif (x,y,direct) in STEPS_PROCESSED:
        #we've already walked through this cell in this direction. Nothing to do
        if DIAG: print("already seen")
        pass
    else:
        # mark cell as hot
        outMap[x,y]=1
        
        STEPS_PROCESSED.add((x,y,direct))
        # STEPS_TO_PROCESS.REMOVE((x,y,direct)) it's already been popped
        if DIAG: print("processing")
        next_steps = decideNextStep(inMap[x,y],direct)
        if DIAG: print("  Have some next steps: ",next_steps)
        for step in next_steps:
            nn = (x+step[0],y+step[1],step)
            if nn in STEPS_PROCESSED:
                pass
            else:
                STEPS_TO_PROCESS.add(nn)
                    
    return outMap

In [103]:
def processMap(inMap,starting_step):
    #print(mirror_map)
    energ_map = np.zeros(inMap.shape)    

    STEPS_PROCESSED  = set([])
    STEPS_TO_PROCESS  = set([starting_step])
    counter = 0 
    while len(STEPS_TO_PROCESS) > 0 and counter < 10000000:
        move = STEPS_TO_PROCESS.pop()
        if DIAG: print("Outer loop: ", move, "counter: ",counter)
        energ_map= processStep(mirror_map,energ_map,*move,STEPS_PROCESSED,STEPS_TO_PROCESS)
    
        counter+=1
    
    res = sum(sum(energ_map))
    if DIAG: print(res)
    return res
    


In [104]:
def getInitalisedDistMap(tgtMap):
    tmpArr = np.ones(tgtMap.shape)*np.inf
    
    return  tmpArr

In [105]:
def neigbours(x,y,mirror_map):
    shX,shY = mirror_map.shape
    dd = [(x-1,y),(x+1,y),(x,y-1),(x,y+1)]
    res = []
    for dx,dy in dd:
        if dx >= 0 and dx<shX and dy>=0 and dy<shY:
            res += [(dx,dy)]
    return res
neigbours(1,0,mirror_map)

[(0, 0), (2, 0), (1, 1)]

In [106]:
def getNewNeighbors(x,y,mirror_map,dist_map):
    # each of neighbors
    # is it a garden space
    # has it alreayd been visited
    res = [] 
    candidates = neigbours(x,y,mirror_map)
    for px,py in candidates:
        if mirror_map[px,py] == CHAR_MAP["#"]:
            continue # a rock
        if dist_map[px,py] == np.inf:
            res += [(px,py)]
    return res

getNewNeighbors(3,4,mirror_map,getInitalisedDistMap(mirror_map))
    
    

[(2, 4), (3, 3), (3, 5)]

In [107]:
def  fillDistancesUpTo(mirror_map,dist_map,max_dist,orig):
    dist=0
    points_at_curr_dist = [orig]
    while dist<=max_dist:
        new_points = []
        for x,y in points_at_curr_dist:
            dist_map[x,y] = dist
            # add neighboring points
            new_points += getNewNeighbors(x,y,mirror_map,dist_map)
        dist+=1
    return dist_map

In [108]:
def select_even_ones(dist_map):
    res_tmp = np.where(dist_map %2 == 0)
    if len(res_tmp) > 1:
        x_list,y_list = res_tmp
        if DIAG: print(x_list,y_list)
    
        res = [(x_list[i],y_list[i]) for i in range(0,len(x_list))]
    else:
        res = []
    return res
    


In [109]:
res = select_even_ones(mirror_map)
#res

In [111]:
import sys


TARGET = 1000000000

np.set_printoptions(threshold=sys.maxsize)

mirror_map,st_point = parseFile(FILE_NAME)
dist_map =getInitalisedDistMap(mirror_map)

max_dist = 1
dist_map = fillDistancesUpTo(mirror_map,dist_map,max_dist,st_point)
list_reachable = select_even_ones(dist_map)

print(mirror_map)
print(st_point)
print(dist_map)
print(list_reachable)

[[0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 1 1 0 1 0]
 [0 1 1 1 0 1 1 0 0 1 0]
 [0 0 1 0 1 0 0 0 1 0 0]
 [0 0 0 0 1 0 1 0 0 0 0]
 [0 1 1 0 0 0 1 1 1 1 0]
 [0 1 1 0 0 1 0 0 0 1 0]
 [0 0 0 0 0 0 0 1 1 0 0]
 [0 1 1 0 1 0 1 1 1 1 0]
 [0 1 1 0 0 1 1 0 1 1 0]
 [0 0 0 0 0 0 0 0 0 0 0]]
(array([5], dtype=int64), array([5], dtype=int64))
[[inf inf inf inf inf inf inf inf inf inf inf]
 [inf inf inf inf inf inf inf inf inf inf inf]
 [inf inf inf inf inf inf inf inf inf inf inf]
 [inf inf inf inf inf inf inf inf inf inf inf]
 [inf inf inf inf inf inf inf inf inf inf inf]
 [inf inf inf inf inf  1. inf inf inf inf inf]
 [inf inf inf inf inf inf inf inf inf inf inf]
 [inf inf inf inf inf inf inf inf inf inf inf]
 [inf inf inf inf inf inf inf inf inf inf inf]
 [inf inf inf inf inf inf inf inf inf inf inf]
 [inf inf inf inf inf inf inf inf inf inf inf]]
[]


C:\Users\Ola\AppData\Local\Temp\ipykernel_16328\2234216733.py:2: RuntimeWarning: invalid value encountered in remainder
  res_tmp = np.where(dist_map %2 == 0)
